In [19]:
import pandas as pd
import numpy as np
import docplex
from docplex.mp.model import Model
from docplex.mp.progress import *
import time
import sys
import math
from sklearn.model_selection import train_test_split

def DataPreperation(datasetname,sampleAmount, featCART, seed):
    global numericalGroup
    numericalGroup = []
    global data
    global trainData
    global samplePos
    global trainDataDummies
    global groupsOfFeatures
    global featureToGroup
    global testData
    global testDataDummies
    global samplePosTest
    # Read in prefered dataset
    if(datasetname == 'chess'):
        dataChess = pd.read_csv("kr-vs-kp.data")
        data = dataChess.dropna().sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns=['won'])
        samplePos = (data['won'] == 'won')*1
    elif(datasetname == 'adult'):
        dataAdult = pd.read_csv("a1a.txt", delim_whitespace=True, header=None) 
        for i in range(1, 15):
            dataAdult[i] = dataAdult[i].str.replace(':1', '', regex=True)
        data = dataAdult.dropna() #.sample(n=sampleAmount, random_state=1)
        data, testData = train_test_split(data, train_size=sampleAmount, random_state=seed)
        
        samplePos = (data[0] == 1)*1
        trainData = data.drop(columns=0)
        
        samplePosTest = (testData[0] == 1)*1
        testData = testData.drop(columns=0)
        
        #numericalGroup = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
    elif(datasetname == 'breast'):
        dataBreast = pd.read_csv("breast.txt", delim_whitespace=True, header=0, names=['Class','Sample code number','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses'])
        dataBreast = dataBreast.drop(columns='Sample code number') #drop id-numbers of observations
        if featCART:
            dataBreast = dataBreast[['Class', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Bare Nuclei']]
        data = dataBreast.sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns='Class')
        for i in trainData.columns: #take away the rownumber: of all variables.
                trainData[i] = trainData[i].str[trainData.iloc[0][i].index(':')+1:]
        samplePos = (data['Class'] == 2)*1
        numericalGroup = [1, 2, 3, 4, 5, 6, 7, 8, 9]
      
    elif(datasetname == 'mushroom'):
        dataMushroom = pd.read_csv("agaricus-lepiota.data", header=0, names=['edible','cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])
        dataMushroom = dataMushroom.replace("?", np.nan)
        data = dataMushroom.dropna()#.sample(n=sampleAmount, random_state=1)
        data, testData = train_test_split(data, train_size=600,random_state=seed)

        #data = dataMushroom.dropna().sample(n=5, random_state=1).drop(columns=['cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])
        
        trainData = data.drop(columns='edible')
        samplePos = (data['edible'] == 'e')*1
        
        samplePosTest = (testData['edible'] == 'e')*1
        testData = testData.drop(columns='edible')
    elif(datasetname == 'tictactoe'):
        dataTicTac = pd.read_csv("tic-tac-toe.data", header=None)
        data = dataTicTac.dropna().sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns=9)
        samplePos = (data[9] == 'positive')*1
    elif(datasetname == 'monks'):
        dataMonks = pd.read_csv("monks-1.test", delim_whitespace=True, header = None)
        data = dataMonks.sample(n=sampleAmount, random_state=1)
        for i in data.columns:
            data[i] = data[i].apply(str)
        trainData = data.drop(columns=[0, 7])
        samplePos = (data[0] == '1')*1
    elif(datasetname == 'votes'):
        dataVotes = pd.read_csv("house-votes-84.data", header=None)
        dataVotes = dataVotes.replace("?", np.nan)
        data = dataVotes.dropna().sample(n=sampleAmount, random_state=1)
        trainData = data.drop(columns=0)
        samplePos = (data[0] == 'democrat')*1
    elif(datasetname == 'heart'):
        dataHeart1 = pd.read_csv("SPECT.test", header=None)
        dataHeart2 = pd.read_csv("SPECT.train", header=None)
        frames = [dataHeart1, dataHeart2]
        dataHeart = pd.concat(frames)
        data = dataHeart.dropna().sample(n=sampleAmount, random_state=1)
        for i in data.columns:
            data[i] = data[i].apply(str)
        trainData = data.drop(columns=0)
        samplePos = (data[0] == '1')*1
    elif(datasetname == 'student'):
        studentMat = pd.read_csv("student-mat.csv", sep=";")
        studentPor = pd.read_csv("student-por.csv", sep=";")
        studentall = pd.merge(studentMat, studentPor, on=["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"])
        numericalGroup = [31, 30, 29, 28, 3]
    else:
        raise ValueError('The dataset was not recognised.')

    #process data for tree

    start = 0
    for i in trainData.columns:
        if (start == 0):
            trainDataDummiesTemp = pd.get_dummies(trainData[i], prefix=i)
            trainDataDummies = trainDataDummiesTemp[sorted(trainDataDummiesTemp.columns.tolist(), key=str.lower)]
            start = 1
        else:
            trainDataDummiesTemp = pd.get_dummies(trainData[i], prefix=i)
            trainDataDummies = trainDataDummies.join(trainDataDummiesTemp[sorted(trainDataDummiesTemp.columns.tolist(), key=str.lower)])
    
    start = 0
    for i in testData.columns:
        if (start == 0):
            testDataDummiesTemp = pd.get_dummies(testData[i], prefix=i)
            testDataDummies = testDataDummiesTemp[sorted(testDataDummiesTemp.columns.tolist(), key=str.lower)]
            start = 1
        else:
            testDataDummiesTemp = pd.get_dummies(testData[i], prefix=i)
            testDataDummies = testDataDummies.join(testDataDummiesTemp[sorted(testDataDummiesTemp.columns.tolist(), key=str.lower)])

    
    groupsN = trainData.shape[1]
    G = range(1, groupsN + 1)
    groupsOfFeatures = trainData.nunique()
    featureToGroup = []
    for g in G:
        amountOfFeatures = groupsOfFeatures.iloc[g-1]
        for f in range(amountOfFeatures):
            featureToGroup.append(g)

In [20]:
def decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity):
    model = Model("tree")
    model.parameters.randomseed = 42
    
    #datashape
    N = trainData.shape[0] # amount of samples
    I = range(1, N + 1) # all samples
    groupsN = trainData.shape[1] # amount of groupes
    groupsOfFeatures = trainData.nunique() # amount of features per group
    G = range(1, groupsN + 1) #groups
    J = range(1, groupsOfFeatures.sum() + 1) #features
    Iplus = np.where(samplePos == 1)[0]
    Iplus = [x+1 for x in Iplus]
    Imin = np.where(samplePos == 0)[0]
    Imin = [x+1 for x in Imin]
    
    
    #tree topology
    if (typeOfTree == 1):
        #simple tree of depth 2
        dNodesN = 3
        lNodesN = 4
        Bplus = [1, 3]
        Bmin = [2, 4]
        route = np.array([[1, 1, 0], [1, -1, 0], [-1, 0, 1], [-1, 0, -1]])
        nonAdjLeaf = [1]
        nonAdjLeafSymm = [1]
        AdjLeaf = [2, 3]
    if(typeOfTree == 2):
        #binary tree with depth 2,5
        dNodesN = 5
        lNodesN = 6
        Bplus = [1, 3, 5]
        Bmin = [2, 4, 6]
        route = np.array([[1, 1, 1, 0, 0], [1, 1, -1, 0, 0], [1, -1, 0, 1, 0], \
                          [1, -1, 0, -1, 0], [-1, 0, 0, 0, 1],  [-1, 0, 0, 0, -1]])
        nonAdjLeaf = [1, 2]
        nonAdjLeafSymm = [2]
        AdjLeaf = [3, 4, 5]
    if(typeOfTree == 3):
        #binary tree with depth 3
        dNodesN = 7
        lNodesN = 8
        Bplus = [1, 3, 5, 7]
        Bmin = [2, 4, 6, 8]
        route = np.array([[1, 1, 1, 0, 0, 0, 0], [1, 1, -1, 0, 0, 0, 0], [1, -1, 0, 1, 0, 0, 0], \
                         [1, -1, 0, -1, 0, 0, 0], [-1, 0, 0, 0, 1, 1, 0], [-1, 0, 0, 0, 1, -1, 0], \
                         [-1, 0, 0, 0, -1, 0, 1], [-1, 0, 0, 0, -1, 0, -1]])
        nonAdjLeaf = [1, 2, 5]
        nonAdjLeafSymm = [1, 2, 5]
        AdjLeaf = [3, 4, 6, 7]
    if(typeOfTree == 4):
        #binary tree with depth 3,5
        dNodesN = 7
        lNodesN = 8
        Bplus = [1, 3, 5, 7]
        Bmin = [2, 4, 6, 8]
        route = np.array([[1, 1, 1, 1, 0, 0, 0], [1, 1, 1, -1, 0, 0, 0], [1, 1, -1, 0, 1, 0, 0], \
                         [1, 1, -1, 0, -1, 0, 0], [1, -1, 0, 0, 0, 1, 0], [1, -1, 0, 0, 0, -1, 0], \
                         [-1, 0, 0, 0, 0, 0, 1], [-1, 0, 0, 0, 0, 0, -1]])
        nonAdjLeaf = [1, 2, 3]
        nonAdjLeafSymm = [3]
        AdjLeaf = [4, 5, 6, 7]
        
    #make nodes
    K = range(1, dNodesN + 1) #decision nodes
    B = range(1, lNodesN + 1) #leaf nodes

    # --- decision variables --- 
    idv = [(g, k) for g in G for k in K]
    if(relax): 
        v = model.continuous_var_dict(keys=idv, lb=0, ub=1, name="V") #relaxation of variables of paragraph 4.3.4
    else:
        v = model.binary_var_dict(keys=idv, name="V")
    idz = [(j,k) for j in J for k in K]
    if(relax):
        idzAdjLeaf = [(j,k) for j in J for k in AdjLeaf]
        idzNonAdjLeaf = [(j,k) for j in J for k in nonAdjLeaf]
        z = model.continuous_var_dict(keys=idzAdjLeaf, lb=0, ub=1, name="Z")
        zNonAdjLeaf = model.binary_var_dict(keys=idzNonAdjLeaf, name="Z")
        z.update(zNonAdjLeaf)
    else:
        idz = [(j,k) for j in J for k in K]
        z = model.binary_var_dict(keys=idz, name="Z")
    idc = [(b,i) for b in B for i in I]
    if(relax):
        c = model.continuous_var_dict(keys=idc,ub=1, name="C")
    else:
        c = model.binary_var_dict(keys=idc, name="C")
    idL = [(i,k) for i in I for k in K]
    L = model.binary_var_dict(keys=idL, name="L")
    idR = [(i,k) for i in I for k in K]
    R = model.binary_var_dict(keys=idL, name="R")
    if(numerical):
        idw = [(g,k) for k in K for g in numericalGroup]
        w = model.binary_var_dict(keys=idw, name="W")
    if(combCon):
        if(numerical):
            idw = [(g,k) for k in K for g in G if ((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard))]
            wcomb = model.binary_var_dict(keys=idw, name="W")
            w.update(wcomb)
        else:
            idw = [(g,k) for k in K for g in G if ((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard))]
            w = model.binary_var_dict(keys=idw, name="W")

    # --- constraints ---
    for k in K:
        model.add_constraint(model.sum(v[g, k] for g in G) == 1) #constraint (1)

    for k in K:
        for j in J:
            model.add_constraint(z[j,k] <= v[featureToGroup[j-1], k]) #constraint (2)

    for k in K:
        for i in I:
            model.add_constraint(L[i,k] == sum(z[j,k] for j in J if trainDataDummies.iloc[i-1,j-1] == 1)) #constraint (3)
            model.add_constraint(R[i,k] == 1 - L[i,k]) #constraint (4)
    
    if(strength):
        for i in I:
            for k in K:
                model.add_constraint(sum(c[b,i] for b in B if route[b-1, k-1] == 1) <= L[i,k]) #constraint (11)
                model.add_constraint(sum(c[b,i] for b in B if route[b-1, k-1] == -1) <= R[i,k]) #constraint (12)
    else:
        for b in B:
            for i in I:
                for k in K:
                    switch = route[b-1, k-1]
                    if switch == 1:
                        model.add_constraint(c[b,i] <= L[i,k]) #constraint (5)
                    elif switch == -1:
                        model.add_constraint(c[b,i] <= R[i,k]) #constraint (6)
        for i in I:
            model.add_constraint(model.sum(c[b,i] for b in B) == 1) #constraint (7)
    
    if(anchor): 
        for g in G:
            for k in nonAdjLeafSymm: #only for nonleafs with symmatrical subtree
                model.add_constraint(z[featureToGroup.index(g) + 1, k] == v[g,k]) #constraint (13)
                        
    if(ExclSameBranchFollowing):
        #for k in K:
        test=1
            #add constraint (9) and (10)
            
    if(numerical):
        #numerical value constraint 4.4
        for k in K:
            for g in numericalGroup:
                featureList = [i for i, x in enumerate(featureToGroup) if x == g]
                for j in featureList:
                    j = j + 1 #making index start at 1
                    if(j-1 == featureList[0]):    
                        model.add_constraint(z[j,k] >= z[j+1,k] - w[g,k])
                    elif(j-1 == featureList[-1]):
                        model.add_constraint(z[j,k] >= z[j-1,k] - (1 - w[g,k]))
                    else:
                        model.add_constraint(z[j,k] >= z[j+1,k] - w[g,k])
                        model.add_constraint(z[j,k] >= z[j-1,k] - (1 - w[g,k]))                     
                
    if(combCon):
        for k in K:
            for g in G:
                if((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard)):
                    featureList = [i for i, x in enumerate(featureToGroup) if x == g]
                    model.add_constraint(model.sum(z[j+1,k] for j in featureList) <= maxCard + (groupsOfFeatures.iloc[g-1] - maxCard)*(1-w[g,k]))
                    model.add_constraint(model.sum(z[j+1,k] for j in featureList) >= (groupsOfFeatures.iloc[g-1] - maxCard) - (groupsOfFeatures.iloc[g-1] - maxCard)*w[g,k])
        
        
    
    # --- objective ---
    model.objectiveTruePos = model.sum(model.sum(c[b,i] for b in Bplus) for i in Iplus)
    model.objectiveTrueNeg = model.sum(model.sum(c[b,i] for b in Bmin) for i in Imin)
    
    if(sensOn):
        
        model.add_constraint(model.sum(model.sum(c[b,i] for b in Bplus) for i in Iplus) >= math.ceil((1-sensitivity)*len(Iplus) ))
        
        totalObjective = model.objectiveTrueNeg
        
        sensitivity
    elif(specOn):
        model.add_constraint(model.sum(model.sum(c[b,i] for b in Bmin) for i in Imin) >= math.ceil((1-specificity)*len(Imin) ))

        totalObjective = model.objectiveTruePos
    else:
        totalObjective = model.objectiveTruePos + C*model.objectiveTrueNeg 
    
    model.maximize(totalObjective)
    
    return model

In [21]:
def treeValidation(solution, testData, trainData):
    error = False
    currentDecNode = 1
    decNodeFeatureDoubleList = []
    decNodeFeatureList = []
    while(not error):
        for feature in range(1, len(trainDataDummies.columns) + 1):
            try:
                value = sol.get_value('Z_' + str(feature) + '_' + str(currentDecNode))
                if(value > 0):
                    decNodeFeatureList.append(feature)
            except:
                error = True
                break
        if (not error):
            currentDecNode = currentDecNode + 1
            decNodeFeatureDoubleList.append(decNodeFeatureList)
            decNodeFeatureList = []
    

    dfTest = pd.DataFrame(columns=trainDataDummies.columns)

    for col in dfTest.columns:
        if col in testData.columns: # copy columns from testdata to dataframe
            dfTest[col] = testData[col]
    dfTest = dfTest.fillna(0) #works
    
    #first part done
    
    #now find good route
    LikArray = np.zeros((len(dfTest.index), len(decNodeFeatureDoubleList)))

    for i in range(len(dfTest)):
        for k in range(len(decNodeFeatureDoubleList)):
            #check if Lik equals 1 or 0
                if(not k == 2):
                    for feature in decNodeFeatureDoubleList[k]:
                        if(dfTest.iloc[i,feature-1] == 1):
                            LikArray[i,k] = 1
    
    
    totalCorrectClassified = 0
    if(typeOfTree == 1):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] == 1):
                if(LikArray[i,2-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
            else:
                if(LikArray[i,3-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
        
        
    elif(typeOfTree == 3):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] == 1):
                if(LikArray[i,2-1] == 1):
                    if(LikArray[i,3-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                else:
                    if(LikArray[i,4-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
            else:
                if(LikArray[i,5-1] == 1):
                    if(LikArray[i,6-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                else:
                    if(LikArray[i,7-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                    
                
    else:
        print('error treetype not found...')
        
    return totalCorrectClassified / samplePosTest.size
    

In [22]:
solTime = 60*60*1 #1 hour
typeOfTree = 3
seed = 1
C = 1  #weight to objective function
ExclSameBranchFollowing = False #constraint (9) and (10)
combCon = False
maxCard = 1
sensOn = False
specOn = False
sensitivity = 0.95
specificity = 0.95
numerical = False

trainacc = []
testacc = []

datasets = [['adult', 100], ['breast', 300], ['chess', 400], ['mushroom', 500], ['tictactoe', 300]]
settings = [[False, False, False], [True, False, True], [True, True, False], [False, True, True], [True, True, True]]
results = []


for dataset in datasets:
    for setting in settings:
        setname = dataset[0]
        samplesize = dataset[1]
        
        DataPreperation(setname, samplesize, False, seed)
        strength = setting[0]
        anchor = setting[1]
        relax = setting[2]
        model = decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity)
        model.set_time_limit(solTime)
        sol = model.solve()
        #gather results
        solveTime = model.solve_details.time
        nodesProcessed = model.solve_details.nb_nodes_processed
        print('Solution for ' + dataset[0] + ' with configuration ' + 'strenght = ' + str(setting[0]) + ' anchor = ' + str(setting[1]) + ' relax = ' + str(setting[2]))
        print(solveTime)
        print(nodesProcessed)
    


Solution for adult with configuration strenght = False anchor = False relax = False
3600.030999999959
131563
Solution for adult with configuration strenght = True anchor = False relax = True
983.125
111278
Solution for adult with configuration strenght = True anchor = True relax = False
584.1870000000345
113775
Solution for adult with configuration strenght = False anchor = True relax = True
1582.3279999999795
108599
Solution for adult with configuration strenght = True anchor = True relax = True
394.655999999959
44543
Solution for breast with configuration strenght = False anchor = False relax = False
1603.9220000000205
21106
Solution for breast with configuration strenght = True anchor = False relax = True
3600.0470000000205
39638
Solution for breast with configuration strenght = True anchor = True relax = False
1019.5
24545
Solution for breast with configuration strenght = False anchor = True relax = True
3600.0470000000205
22867
Solution for breast with configuration strenght = Tru